In [ ]:
import bs4
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import requests

import datetime
import os
import csv

# 1. Data Collection

In [36]:
#mpp_url = 'https://www.atlasobscura.com/places?page={}&sort=likes_count'.format(i)
# write first 400 pages places_urls to txt_file_urls.txt
#

url_name = 'https://www.atlasobscura.com'
for i in range(400):
    result = requests.get('https://www.atlasobscura.com/places?page={}&sort=likes_count'.format(i+1))
    list_soup = bs(result.text)
    list_places = [x.get('href') for x in list_soup.find_all('a', {'class':'content-card-place'})]
    with open ('txt_file_urls.txt','a' ) as f:
        for el in list_places:
            f.write(url_name+el +'\n')

In [568]:
# for each url get html and store it in /htmls_page_{page_nr}/{place_name}
#

def url_to_text_(urls):
    i = urls[0]
    url = urls[1].split('\n')[0]
    try:
        os.makedirs('htmls_page_{}'.format(i+1))
    except:
        pass
    el = url
    el_name = el.split('.com/places/')[1]
    result = requests.get(el)
    with open('./htmls_page_{}/{}'.format(i+1, el_name), 'w') as wf:
        wf.write(result.text)


In [567]:
# looping over urls in groups of 18 (nr places per site) call urls_to_text_() function
#

if __name__ == '__main__':
    with open ('txt_file_urls.txt', 'r') as urls:
        l = []
        for line in urls:
            l.append(line)
        grouped_l = [l[i:i+18] for i in range(0,len(l), 18)]
        grouped_ln = [[(i, el) for el in group]for i, group in enumerate(grouped_l)]

        grouped_ln_flat = []
        for group in grouped_ln:
            for el in group:
                grouped_ln_flat.append(el)
         
        for el in grouped_ln_flat[:]:
            url_to_text_(el)       

In [ ]:
# for page_nr, place_name extract information from html, if NA set ==''
# write to tsvs_page_{page_nr}/{place_name}.tsv

def analyse_html(page_nr, name, to_tsv = True): 
    
    with open('./htmls/htmls_page_{}/{}'.format(page_nr,name), 'r') as f:
        soup = bs(f.read())
    
    try:
        placeName = str(soup.find_all('h1', {'class':'DDPage__header-title'})[0].contents[0])
    except:
        placeName =''

    try:
        a = soup.find_all('a', {'class': 'itemTags__link js-item-tags-link'})
        placeTags = [el.get('href').split('/categories/')[1] for el in a]
    except:
        placeTags =''

    try:
        nrs = soup.find_all('div', {'class':'title-md item-action-count'})[:2]
        numPeopleVisited = int(nrs[0].get_text())
    except:
        numPeopleVisited=''

    try:
        nrs = soup.find_all('div', {'class':'title-md item-action-count'})[:2]
        numPeopleWant = int(nrs[1].get_text())
    except:
        numPeopleWant = ''

    try:
        place_desc = soup.find_all('div', {'class': 'DDP__body-copy'})
        text = ''.join([el.get_text() for el in place_desc])
        placeDesc = text.replace('\n', '').replace('\xa0', '')
    except:
        placeDesc=''

    try:
        placeShortDesc = soup.find_all('h3', {'class': 'DDPage__header-dek'})[0].contents[0]
    except:
        placeShortDesc=''

    try:
        side = soup.find_all('div', {'class':'DDPageSiderailRecirc__item-title'})
        placeNearby = [s.get_text() for s in side]
    except:
        placeNearby = ''

    try:
        a=soup.find_all('address', {'class':'DDPageSiderail__address'})[0]
        placeAdress = a.find('div').get_text(separator = ' ').split('\n')[0]
    except:
        placeAdress = ''

    try:
        b=soup.find('div', {'class':'DDPageSiderail__coordinates js-copy-coordinates'})
        lat,lng = b.get('data-coordinates').split(',')
        placeAlt = float(lat)
        placeLong = float(lng)
    except:
        placeAlt = ''
        placeLong = ''


    try:
        contributors = soup.find_all('div', {'class': 'ugc-editor-icons'})
        href = [c.findChildren('a')[0].get('href') for c in contributors]
        #alt = [c.findChildren('img')[0].get('alt') for c in contributors]
        placeEditors = [h.split('/users/')[1] for h in href]
    except:
        placeEditors = ''

    try:    
        date = soup.find_all('div', {'class': 'DDPContributor__name'})
        dt_string = date[0].get_text()
        format = "%B %d, %Y"
        placePubDate = datetime.datetime.strptime(dt_string, format)
    except:
        placePubDate= ''

    try:
        pot = soup.find_all('div', {'class':'CardRecircSection__title'})
        lists = [o for o in pot if o.text == 'Appears in 1 List'][0]
        pl = lists.find_all_next('h3', {'class':'Card__heading --content-card-v2-title js-title-content'})
        placeRelatedLists = [el.find('span').text for el in pl]
    except:
        placeRelatedLists = ''

    try:
        pot = soup.find_all('div', {'class':'CardRecircSection__title'})
        rel = [o for o in pot if o.text == 'Related Places'][0]
        pl = rel.find_all_next('div', {'class':'Card__action-btns vue-js-been-there-everywhere-place'})
        placeRelatedPlaces = [p.get('data-place-title') for p in pl]
    except:
        placeRelatedPlaces=''

    try:
        placeURL = 'https://www.atlasobscura.com/places/'+name
    except:
        placeURL = ''
        
    info = [placeName, placeTags, numPeopleWant, numPeopleVisited, placeDesc, placeShortDesc, placeNearby, placeAdress, placeAlt, placeLong, placeEditors, placePubDate, placeRelatedLists, placeRelatedPlaces, placeURL]
   
    if to_tsv:
        with open ('tsvs_page_{}/{}.tsv'.format(i+1, name), 'w') as f:
            writer = csv.writer(f, delimiter='\t')
            writer.writerow(info)
            

In [ ]:
# loop through html files and extract relevant data
#

for i in range(400):
    els = os.listdir('./htmls/htmls_page_{}'.format(i+1))
    os.makedirs('tsvs_page_{}'.format(i+1))

    for name in els:
        analyse_html(i+1, name)

# 2. Search Engine

In [510]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import re

from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer


# function takes a description in string format, consecutively does:
# to lower case, remove stopwords, remove punctutaion, stemming, removing words with len()<=1

def description_to_stems(desc):
    
    lower = desc.lower()
    
    stopword = set(stopwords.words('english'))

    word_tokens = RegexpTokenizer('\s', gaps=True).tokenize(lower)

    regex = r"[!\"#\$%&\“\”\'\(\)\*\+,-\./:;<=>\?@\[\\\]\^_`{\|}~]"
    no_stops = [re.sub(regex, '', w) for w in word_tokens if not w in stopword]

    stemmer = SnowballStemmer('english')
    stems = [stemmer.stem(w) for w in no_stops if len(stemmer.stem(w))>1]

    words = stems
    return words

In [427]:
# looping pages and places.tsv; read the description and pass to the preprocessing function description_to_stems()
# sequence of words returned that are stored in a dictionary, that maps str -> nr of elements added before (lenght of dict)

vocabulary = {}

for i in range(400):
    els = os.listdir('./tsvs/tsvs_page_{}'.format(i+1))

    for name in tqdm(els):
        with open ( './tsvs/tsvs_page_{}/{}'.format(i+1, name), 'r') as f:
            tsv_reader = csv.reader(f, delimiter="\t")
            for row in tsv_reader:# if i ==4][0]
                desc = row[4]
                words = set(description_to_stems(desc))
                
                for word in words:
                    if word not in vocabulary.keys():
                        vocabulary[word] = int(len(vocabulary))

100%|██████████████████████████████████████████| 18/18 [00:00<00:00, 174.45it/s]


100%|██████████████████████████████████████████| 18/18 [00:00<00:00, 188.60it/s]


100%|██████████████████████████████████████████| 18/18 [00:00<00:00, 161.61it/s]


100%|██████████████████████████████████████████| 18/18 [00:00<00:00, 181.39it/s]


In [432]:
# saving the vocabulary dictionary to csv

with open("vocabulary.csv", "w") as voc_f:
    w = csv.writer(voc_f)
    for key, val in vocabulary.items():
        w.writerow([key, val])


In [512]:
# preprocessing the descriptions, map documents words to index and generate a dict
# of index -> page_index/documentname.tsv
mapping = {}

for i in range(400):
    els = os.listdir('./tsvs/tsvs_page_{}'.format(i+1))

    for name in tqdm(els):
        with open ( './tsvs/tsvs_page_{}/{}'.format(i+1, name), 'r') as f:
            tsv_reader = csv.reader(f, delimiter="\t")
            for row in tsv_reader:# if i ==4][0]
                desc = row[4]
                words = set(description_to_stems(desc))
                for word in words:
                    idx = vocabulary[word]
                    try:
                        mapping[idx].append(str(i+1)+'/'+name) 
                    except:
                        mapping[idx] = [str(i+1)+'/'+name]

100%|██████████████████████████████████████████| 18/18 [00:00<00:00, 185.85it/s]


100%|██████████████████████████████████████████| 18/18 [00:00<00:00, 202.28it/s]


100%|██████████████████████████████████████████| 18/18 [00:00<00:00, 176.07it/s]


100%|██████████████████████████████████████████| 18/18 [00:00<00:00, 207.97it/s]


In [458]:
# saving the term_id to document_name dictionary to csv

with open("termid_document.csv", "w") as map_f:
    w = csv.writer(map_f)
    for key, val in mapping.items():
        w.writerow([key, val])

In [441]:
inp = str(input())

museum american horror


In [454]:
# given a string input with blank space each word is mapped to the vocabulary index, which is then mapped to
# the matching documents by the termid_document dict. 
# the intersection of matching document sets is determined and the document names returned

def docs_matching_query(inp):
    sub_matches = []
    for el in inp.split(' '):
        try:
            idx = vocabulary[el]
            subm = mapping[idx]
            sub_matches.append(subm)
        except:
            pass    
    try:
        both = set(sub_matches[0])
        for sub_query in sub_matches:
            both = both.intersection(sub_query)
    except:
        both = ''
        
    return both


In [488]:
# print data of the matching documents 
import pandas as pd

docs = docs_matching_query(inp)
dfs = []
for el in docs:
    i, name = el.split('/')
    with open ( './tsvs/tsvs_page_{}/{}'.format(int(i), name), 'r') as f:
        tsv_reader = csv.reader(f, delimiter="\t")
        for row in tsv_reader:# if i ==4][0]
            new_df = pd.DataFrame([row[0], row[4], row[14]]).T
            dfs.append(new_df)

df = pd.concat(dfs)
df.columns=['placeName', 'placeDesc', 'placeURL']
df

,placeName,placeDesc,placeURL
0,Murals of Maxo Vanka,Created in protest against industrial capitali...,https://www.atlasobscura.com/places/murals-of-...
0,Grave of Ed Gein,"While not technically a serial killer, the ter...",https://www.atlasobscura.com/places/plainfield...
0,Buckhorn Saloon and Museum,Drink a beer whilst you admire stuffed cougars...,https://www.atlasobscura.com/places/buckhorn-s...
0,PsychoBarn,If you’re casually strolling down Piccadilly a...,https://www.atlasobscura.com/places/psychobarn


# 2.2?

In [570]:
import numpy as np
from collections import Counter
import pandas as pd

In [496]:
# weights for each word in vocabulary, term_id -> weight_term

weights = {}
n = 7200

for key in mapping.keys():
    weights[key] = np.log(n/len(mapping[key]))

In [521]:
# mapping terms to list of (file_name, tf-idf)
# tf-idf = tfj * wj = tfj * log(n/nj) n = 7200

mapping_tfidf = {}
for i in range(400):
    els = os.listdir('./tsvs/tsvs_page_{}'.format(i+1))

    for name in tqdm(els):
        with open ( './tsvs/tsvs_page_{}/{}'.format(i+1, name), 'r') as f:
            tsv_reader = csv.reader(f, delimiter="\t")
            for row in tsv_reader:
                desc = row[4]
                words = description_to_stems(desc)
                ln = len(words)
                
                counter = Counter(words)
                for word in counter:
                    idx = vocabulary[word]
                    freq = counter[word]/ln*weights[idx] # == tf
                    try:
                        mapping_tfidf[idx].append( (str(i+1)+'/'+name, freq) )
                    except:
                        mapping_tfidf[idx] = [ (str(i+1)+'/'+name, freq) ]

100%|██████████████████████████████████████████| 18/18 [00:00<00:00, 165.56it/s]


100%|██████████████████████████████████████████| 18/18 [00:00<00:00, 179.67it/s]


100%|██████████████████████████████████████████| 18/18 [00:00<00:00, 157.18it/s]


100%|██████████████████████████████████████████| 18/18 [00:00<00:00, 187.91it/s]


In [523]:
# writing freq_mapping to csv

with open("termid_document_freq.csv", "w") as map_ffreq:
    w = csv.writer(map_ffreq)
    for key, val in mapping_tfidf.items():
        w.writerow([key, val])

In [526]:
# testing 

inp = str(input())

matches = docs_matching_query(inp)

american flower


In [527]:
matches

{'124/atocha-station-tropical-garden.tsv',
 '126/fenway-victory-gardens.tsv',
 '189/plainfield-cemetery.tsv',
 '199/medicine-rocks-state-park.tsv',
 '254/felicity-california-center-of-the-world.tsv',
 '27/bannermans-castle.tsv',
 '310/carnivorous-plants-at-the-huntington-gardens.tsv',
 '322/hare-krishna-tree.tsv',
 '353/coltsville.tsv',
 '396/cole-porter-s-piano.tsv',
 '68/harvard-museum-of-natural-history.tsv',
 '78/casina-delle-civete.tsv',
 '79/mueller-sunflowers.tsv',
 '91/dumbarton-oaks.tsv',
 '91/ethel-m-botanical-cactus-garden.tsv'}

In [563]:
# taking input_query, list of matching documents, int k
# read matching document_description, count occurence of the words that are in the input_query/len(description)

# since we only need the relative similarity between matches, the dimensions of the input are irrelevant
# the cosine similarity is ~ to vector_of_words_in_match/len(vector_of_words_in_match)
# since all dimensions but the words of the input query are orthogonal we can look only at the values in 
# that dimension and normalize by the length of the whole vector_of_words_in_match==len(description)

def cos_rel_similarst(inp, matches, k):
    for match in matches:
        i, name = match.split('/')
        with open ( './tsvs/tsvs_page_{}/{}'.format(int(i), name), 'r') as f:
            tsv_reader = csv.reader(f, delimiter="\t")
            for row in tsv_reader:
                desc = description_to_stems(row[4])# if i ==4][0]
                ln = len(desc)
                counter = Counter(desc)
                sim = sum(counter[sub_query] for sub_query in inps)/ln
                if len(k_matches) < k:
                    k_matches[sim] = ([row[0], row[4], row[14]])
                else:
                    if sim > min( list(k_matches.keys()) ):
                        min_key = min( list(k_matches.keys()) )
                        k_matches.pop(min_key)
                        k_matches[sim] = ([row[0], row[4], row[14]])

    df = pd.DataFrame.from_dict(k_matches, orient='index')
    df = df.reset_index(drop=True)
    df['Similarity'] = list (k_matches.keys() )

    df.columns=['placeName', 'placeDesc', 'placeURL', 'Similarity']
    df = df.sort_values(by='Similarity', ascending = False)
    return df

In [564]:
cos_rel_similarst(inp, matches, 3)

,placeName,placeDesc,placeURL,Similarity
1,Atocha Station Tropical Garden,The winding paths and many benches within this...,https://www.atlasobscura.com/places/atocha-sta...,0.024631
0,Fenway Victory Gardens,"While World War II battles raged overseas, bac...",https://www.atlasobscura.com/places/fenway-vic...,0.019608
2,Mueller SunFlowers,"On a certain stretch of I-35 near Austin, Texa...",https://www.atlasobscura.com/places/mueller-su...,0.018987


# Notes